Import the libraries

In [1]:
import numpy as np
import sympy as sp
import cvxpy as cp
import itertools
from scipy.linalg import null_space
from scipy.linalg import sqrtm
from numpy.linalg import inv
from numpy.linalg import det
from dataclasses import dataclass
from tqdm.notebook import tqdm

The tolerance for numerical computation, set to 1e-10 by default.

In [2]:
a_tol = 1e-10 # The absolute tolerance
r_tol = 1e-7  # The relative tolerance

Data models

In [3]:
@dataclass
class Word_Bis:               # The class of bisectors in the symmetric space $X_3$. A list of Word_Bis models describes the bisectors defining the Dirichlet-Selber domain.
    word: np.ndarray          # A matrix $g$ in $SL(3,R)$, typically a word in given generators
    bis: np.ndarray           # A normal vector (as a 3*3 matrix) of the Selberg bisector $Bis(X, g.X)$

@dataclass
class Poly_Face:              # The class of faces of polytopes in $X_3$. A list of Poly_Face models describes the polytope structure of the Dirichlet-Selber domain.
    equs: list[int]           # A list of bisectors indices (in the list of the accompanying Word_Bis models) whose intersection is the minimal plane containing the face.
    codim: int                # The codimension (5 - dim) of the face.
    subfaces: list[int]       # A list of face indices that are proper subfaces of the current face.
    sample_point: np.ndarray  # A point in $X_3$ (as a 3*3 matrix) lying in the interior of the current face.

@dataclass
class Find_Intersection:      # The class describing if the union of certain $X_3$ hyperplanes is empty.
    sample_point: np.ndarray  # A sample point of the intersection if non-empty, or the zero matrix if empty.
    is_intersection: bool     # Boolean variable describing if the intersection is empty.

@dataclass
class Ridge_Cycle:           # The class describing a ridge-cycle of a Dirichlet-Selberg domain.
    ridge: list[int]          # A list of ridge indices (in the list of the accompanying Poly_Face models), for ridges $r_0, r_1, r_2,...$ in the same ridge cycle.
    pairing: list[int]        # A list of word indices (in the list of the accompanying Word_Bis models), each word $g_i$ sends $r_i$ to $r_{i+1}$.

Generic Helper functions

In [4]:
# Check if the new vector is in the span of a independent set of vectors (with tolerance)
def in_span(vectors, new_vector):
    if len(vectors) == 0:
        return np.linalg.norm(new_vector) < a_tol               # The first vector (if nonzero) is independent on its own
    matrix = np.array(vectors).T                              # Stack the current independent vectors into a matrix, so each column is a vector    
    projection = matrix @ np.linalg.pinv(matrix) @ new_vector # Compute the projection of the new vector onto the space spanned by the existing vectors
    residual = new_vector - projection                        # Compute the difference (residual) between the new vector and its projection
    return np.linalg.norm(residual) < a_tol                     # The new vector is dependent to the existing ones if the residual is smaller than the threshold

# Return a linearly independent subset of vectors (with tolerance)
def extract_basis(vectors):
    indep_vectors = []
    for vector in vectors:
        if not in_span(indep_vectors, vector):
            indep_vectors.append(vector)
    return indep_vectors

# Check if the vector tuples span the same subspace
def same_span(vectors_A, vectors_B):
    rank_A = len(extract_basis(vectors_A))                   # Check if they define the same plane by a rank argument
    rank_B = len(extract_basis(vectors_B))
    rank_AplusB = len(extract_basis(vectors_A + vectors_B))
    return (rank_A == rank_AplusB) and (rank_B == rank_AplusB)
    
# Check if a symmetric matrix (in np.array) is positive definite. Return true if it is, false if it may not be (concerning the tolerance).
def is_positive_definite(matrix):
    if not isinstance(matrix, np.ndarray):
        raise ValueError("Input must be a numpy array.")
    if matrix.shape[0] != matrix.shape[1]:
        return False                                  # The matrix is not square
    try:
        min_diag = np.min(np.linalg.eigvalsh(matrix)) # Find the smallest eigenvalue
        return min_diag > a_tol                         # Positive definite if it is positive (concerning the tolerance).
    except np.linalg.LinAlgError:
        return False                                  # Not positive definite

# Converting between symmetric 3*3 matrices and 6-dimensional vectors
def word_to_vector(word, param): #word_to_vector(word, 1)
    vector = [word[0][0], word[1][1], word[2][2], param * word[0][1], param * word[0][2], param * word[1][2]]
    return vector

def vector_to_word(vector, param): #vector_to_word(vector, 1/2)
    word = np.array([[vector[0], param * vector[3], param * vector[4]],
                     [param * vector[3], vector[1], param * vector[5]],
                     [param * vector[4], param * vector[5], vector[2]]])
    return word

# Find the orthogonal complement of 3*3 symmetric matrices, with respect to the product trace.
def orth_matrix(matrices):
    vectors = [word_to_vector(matrix, 1) for matrix in matrices]
    vectors = extract_basis(vectors)
    orth_vectors = null_space(np.array(vectors)).T.tolist()
    orth_matrices = [np.array(vector_to_word(orth_vector, 1/2)) for orth_vector in orth_vectors]
    return orth_matrices

# Convert a matrix of linear expressions in sympy to cvxpy for convex optimization purposes.
def sp_to_cp(M_sym, variables):
    free_syms = variables
    n_vars = len(free_syms)
    rows, cols = M_sym.shape                                                                # Get the matrix shape
    coeff_matrices = [np.zeros((rows, cols), dtype=np.float64) for _ in range(n_vars + 1)]  # Initialize the coefficient matrices
    for i in range(rows):
        for j in range(cols):                 
            expr = sp.expand(M_sym[i, j])                                                   # Decompose the matrix into entries
            if expr.is_Add:                 
                terms = expr.as_ordered_terms()                                             # Convert the entry into a list of summands
            else:
                terms = [expr]                                                              # Simply wrap the entry if it is a single term
            for term in terms:
                found = False
                for idx, sym in enumerate(free_syms):
                    if term.has(sym):
                        coeff = term.coeff(sym)
                        coeff_matrices[idx + 1][i, j] = float(coeff)                        # Add the coefficient of a certain variable to the corresponding matrix
                        found = True
                        break
                if not found:
                    coeff_matrices[0][i, j] = float(term)                                   # Add the constant term to the zeroth matrix
    x_cvx = cp.Variable(n_vars)                                                             # Define the cvxpy variables
    M_cvx = coeff_matrices[0] + sum(x_cvx[i]*coeff_matrices[i+1] for i in range(n_vars))    # Combine the coefficient matrices into a cvxpy matrix with variables
    return M_cvx

Question-specific helper functions

In [5]:
# Compute a basis of vectors perpendicular to a given 3-dimensional vector.
# Specifically, given an indefinite vector, the first output vector is positive definite, while the second output vector is indefinite.
def compute_vector(d):
    positive_indices = [i for i, x in enumerate(d) if x > a_tol]
    negative_indices = [i for i, x in enumerate(d) if x < -a_tol]
    if not positive_indices or not negative_indices:
        return None, None                 # Our program does not focus on definite input vectors
    positive_index = positive_indices[0]  # Find the first positive component index of the vector
    negative_index = negative_indices[0]  # Find the first negative component index of the vector
    d_sorted = [0, 0, 0]                  # Rearrange the components of the vector into (pos, neg, rest)
    d_sorted[0] = d[positive_index]       # The positive one becomes d0
    d_sorted[1] = d[negative_index]       # The negative one becomes d1
    d_sorted[2] = d[3 - positive_index - negative_index] # The remaining becomes d2
    d0, d1, d2 = d_sorted
    if d2<0:                              # Compute the perpendicular vectors in the new order
        v = [-d1/(np.sqrt(d0**2 + d1**2)) - d2/(np.sqrt(d0**2 + d2**2)), d0/(np.sqrt(d0**2 + d1**2)), d0/(np.sqrt(d0**2 + d2**2))]
        w = [-d1/(np.sqrt(d0**2 + d1**2)) + d2/(np.sqrt(d0**2 + d2**2)), d0/(np.sqrt(d0**2 + d1**2)), -d0/(np.sqrt(d0**2 + d2**2))]
    else:
        v = [-d1/(np.sqrt(d0**2 + d1**2)), d0/(np.sqrt(d0**2 + d1**2)) + d2/(np.sqrt(d1**2 + d2**2)), -d1/(np.sqrt(d1**2 + d2**2))]
        w = [-d1/(np.sqrt(d0**2 + d1**2)), d0/(np.sqrt(d0**2 + d1**2)) - d2/(np.sqrt(d1**2 + d2**2)), d1/(np.sqrt(d1**2 + d2**2))]
    v_original_order = [0, 0, 0]          # Permute the vector back to the original order
    v_original_order[positive_index] = v[0]
    v_original_order[negative_index] = v[1]
    v_original_order[3 - positive_index - negative_index] = v[2]
    w_original_order = [0, 0, 0]
    w_original_order[positive_index] = w[0]
    w_original_order[negative_index] = w[1]
    w_original_order[3 - positive_index - negative_index] = w[2]
    return v_original_order, w_original_order

# Use convex optimization to find positive definite combination of certain 3*3 symmetric matrices
def find_pos_def(mat_expr, variables):
    t = cp.Variable()
    l = len(variables)
    x = cp.Variable(l)
    M = sp_to_cp(mat_expr, variables)                     # The combination of certain matrices
    constraints = [M - t*np.eye(3) >> 0, x >= -1, x <= 1] # Our question only concerns coefficients lying between -1 and 1
    prob = cp.Problem(cp.Maximize(t), constraints)        # Find maximal t such that M-tI is positive definite
    prob.solve(solver=cp.SCS,eps=a_tol,max_iters=50000)     # Add verbose=True if needed
    if prob.value > 100*a_tol:
        return M.value                                    # The positive definite linear combination with maximized least eigenvalue
    else:
        return None                                       # No positive definite linear combinations

# Compute the Riemannian angle between two hyperplanes (represented by normal vectors) in X_3 at a certain base point
# The formula is given in my paper
def Riemannian_angle(equ_1, equ_2, mat):
    comp_1 = mat @ equ_1
    comp_2 = mat @ equ_2
    angle_cos = - (np.trace(comp_1 @ comp_2))/(np.sqrt((np.trace(comp_1 @ comp_1)) * (np.trace(comp_2 @ comp_2))))
    angle = np.arccos(angle_cos)
    return angle

# Find a positive definite matrix on the elongation of the line from the first matrix to the second one
def elongate(matrix_1, matrix_2):
    matrix_1 = np.array(matrix_1)
    matrix_2 = np.array(matrix_2)
    if not is_positive_definite(matrix_2):
        raise ValueError("elongate: Input must be positive definite.")
    else:
        matrix = 2*matrix_2 - matrix_1            # The elongation
        while not is_positive_definite(matrix):   # Go back toward matrix_2 if matrix is indefinite
            matrix = 0.5*matrix + 0.5*matrix_2
        matrix = matrix/((det(matrix)) ** (1/3))  # Unitize the matrix with respect to the determinant
        return np.array(matrix)

# The input positive definite matrix lies on the plane defined by some equations as well as a new equation
# Perturb it to the positive side of the hyperplane defined by the new equation while remaining positive definite and lying on the plane defined by the old equations
def perturb_within_plane(matrix, equations, new_equation):
    matrix = np.array(matrix)                                                           # make the matrices numpy for safety reason
    equations = [np.array(equation) for equation in equations]
    new_equation = np.array(new_equation)
    if not is_positive_definite(matrix):
        raise ValueError("perturb_within_plane: Input must be positive definite.")
    else:
        matrix_sqrt = sqrtm(matrix)                                                     # Congruence so the input matrix is taken to the origin
        equations_trans = [matrix_sqrt @ equ @ matrix_sqrt for equ in equations]
        new_equation_trans = matrix_sqrt @ new_equation @ matrix_sqrt
        orth_equations = orth_matrix(equations_trans)
        orth_vectors = [word_to_vector(equation, np.sqrt(2)) for equation in orth_equations]    # Convert from matrices to vectors
        new_vector = word_to_vector(new_equation_trans, np.sqrt(2))
        orth_vectors_matrix = np.array(orth_vectors).T                                  # Stack the current independent vectors into a matrix, each column is a vector
        coeffs = list(np.linalg.pinv(orth_vectors_matrix) @ new_vector)                 # Project the new vector to the existing ones
        projection_trans = sum(coeff*equ for coeff, equ in zip(coeffs, orth_equations)) # This linear combination lies on the desired plane while keeps away from the new hyperplane
        projection = matrix_sqrt @ projection_trans @ matrix_sqrt                       # Take the matrix back
        while not is_positive_definite(projection):
            projection = 0.5*projection + 0.5*matrix                                    # Go back toward the original matrix if the new one is indefinite
        projection = projection/((det(projection)) ** (1/3))                            # Unitize the matrix with respect to the determinant
        return projection

# Check if the word in SL(3,R) takes the old plane (defined by a set of normal matrices) to the new plane
def equal_spaces(old_equations, new_equations, word):
    mapped_equations = [inv(word) @ mat @ inv(word.T) for mat in old_equations] # The normal matrices for the mapped plane
    mapped_vectors = [word_to_vector(mat, 1) for mat in mapped_equations]          # Convert from matrices to vectors
    new_vectors = [word_to_vector(mat, 1) for mat in new_equations]
    return same_span(mapped_vectors, new_vectors)

Special Utilities

In [6]:
# Compute for all distinct words of a given maximal length from given generators in SL(3,R)
def distinct_matrix_products(generators, max_length):
    generators = [np.array(mat) for mat in generators] # Make sure they are numpy arrays
    k = len(generators)
    G = generators + [inv(mat) for mat in generators]
    inv_idx = lambda j: (j + k) % (2*k)
    seen = [np.eye(generators[0].shape[0])]  # The list of matrices and last-index-list, List[np.ndarray]
    frontier = {0: set()} # Dict[int, Set[int]]
    for _ in range(max_length):
        next_frontier = {}  # Dict[int, Set[int]] 
        for ind in frontier:            # A tuple of word and a possible last-index
            for j, mat in enumerate(G):        # A tuple of generator and its index to be added
                if inv_idx(j) in frontier[ind]:
                    continue                   # Avoid backtracking
                N = seen[ind] @ mat                    # If not, add the generator to the end of the word
                matched = False
                for S_ind in range(len(seen)):
                    if np.allclose(N, seen[S_ind], atol = a_tol, rtol = r_tol): # The word is already there; if it is in next_frontier, add a last-index
                        matched = True
                        if S_ind in next_frontier:
                            next_frontier[S_ind].add(j)
                        break
                if not matched:
                    seen.append(N)
                    N_ind = len(seen) - 1
                    next_frontier[N_ind] = set()
                    next_frontier[N_ind].add(j)
        frontier = next_frontier
        if not frontier:
            break
    seen.sort(key=lambda M: np.trace(M.T @ M)) # sort by Frobenius norm
    return seen

# Compute bisectors Bis(X, g.X) (as in dataclass Word_Bis) from a list of generators, a maximal word length and a center X
# Words that stabilize X will be excluded for well-defined bisectors
def word_bisectors(generators, length, center):
    words = distinct_matrix_products(generators, length)
    wbs = [Word_Bis(word, np.array(word) @ inv(np.array(center)) @ np.array(word).T - inv(np.array(center)))\
           for word in words]                                                # Definition of Selberg bisectors
    wbs_filtered = [wb for wb in wbs if not np.all(np.abs(wb.bis)<a_tol)]      # Exclude bisectors with zero normal matrices
    return wbs_filtered

# Determine if the intersection of given hyperplanes is non-empty. Produce a sample point if so.
def find_positive_definite_intersection(words):
    ################## Part 1: Consider a linearly independent sublist
    vectors = [word_to_vector(word, 1) for word in words]
    independent_vectors = extract_basis(vectors)
    indep_matrix = [vector_to_word(vec, 1) for vec in independent_vectors]
    indep_matrix_sp = [sp.Matrix(mat) for mat in indep_matrix]
    ################### Part 2: The non-case
    A = indep_matrix[0]
    eigenvalues, eigenvectors = np.linalg.eigh(A)
    ev_orth, ev_orth_neg = compute_vector(eigenvalues)
    if ev_orth is None:
        sample_point = np.zeros((3, 3)) 
        is_intersection = False
    else:
        n = len(indep_matrix)
        Q = eigenvectors
        D_orth = np.diag(ev_orth)
    ################### Part 3: The case if n = 1
        if n == 1:
            sample_point = Q @ D_orth @ Q.T
            is_intersection = True
    ################### Part 4: The case if n > 1, we will begin canceling the variables
        else:
            linearized_matrix = [Q.T @ mat @ Q for mat in indep_matrix]
            linearized_matrix_sp = [sp.Matrix(mat) for mat in linearized_matrix]
            x_v = sp.symbols('x1:5')
            x_v_new = x_v #The necessary variables
            diag_max = [a + abs(b) for a, b in zip(ev_orth, ev_orth_neg)]
            D_orth_neg = [np.diag(ev_orth_neg),\
                        vector_to_word(np.sqrt(diag_max[0]*diag_max[1]) * np.eye(6)[:, 3], 1),\
                        vector_to_word(np.sqrt(diag_max[0]*diag_max[2]) * np.eye(6)[:, 4], 1),\
                        vector_to_word(np.sqrt(diag_max[1]*diag_max[2]) * np.eye(6)[:, 5], 1)]
            D_orth_sp = sp.Matrix(D_orth)
            D_orth_neg_sp = [sp.Matrix(mat) for mat in D_orth_neg]
            matrix_comb = sum((var * mat for mat, var in zip(D_orth_neg_sp, x_v)), start=D_orth_sp)
            # Remove extra variables
            for i in range(1,n):
                trace_matrix_prod = (matrix_comb * linearized_matrix_sp[i]).trace().expand()
                trace_coeffs = {var: trace_matrix_prod.coeff(var) for var in x_v}
                #If a nonzero constant appears, return to false since it is surely empty.
                if all(abs(coeff) < a_tol for coeff in trace_coeffs.values()):
                    sample_point = np.zeros((3, 3)) 
                    is_intersection = False
                    return Find_Intersection(np.array(sample_point), is_intersection) 
                max_var = max(trace_coeffs, key=lambda v: abs(trace_coeffs[v]))
                x_sol = sp.solve(trace_matrix_prod, max_var)[0]  # Solve f = 0 for max_var
                x_v_new = tuple(var for var in x_v_new if var != max_var) # Drop max_var from x_v_new
                matrix_comb = matrix_comb.subs(max_var,x_sol)
            # If all variables are removed
            if n == 5:
                D = np.array(matrix_comb).astype(np.float64)
                if is_positive_definite(D):
                    sample_point = Q @ D @ Q.T
                    is_intersection = True
                else:
                    sample_point = np.zeros((3, 3)) 
                    is_intersection = False
            # Set the equations
            else:
                poly_comb = matrix_comb.det()
                poly_comb_coeff = poly_comb.as_coefficients_dict()
                all_zero = all(abs(coef) < a_tol for coef in poly_comb_coeff.values())
                if all_zero:
                    sample_point = np.zeros((3, 3)) 
                    is_intersection = False
                else:
                    D = find_pos_def(matrix_comb, x_v_new)
                    if D is None:
                        sample_point = np.zeros((3, 3)) 
                        is_intersection = False
                    else:
                        sample_point = Q @ D @ Q.T
                        is_intersection = True
    if is_intersection:
        if not is_positive_definite(sample_point):
            is_intersection = False
            sample_point = np.zeros((3, 3))
        else:
            sample_point = sample_point/((det(sample_point)) ** (1/3))
    return Find_Intersection(np.array(sample_point), is_intersection)

# Add a new bisector new_wb to the existing polytope in X_3 and compute for the new polytope structure.
# The existing bisectors are described by my_wbs and the polytope structure is described by my_face_list. 
def selberg_domain_add_facet(my_wbs, my_face_list, new_wb):
    new_vec = word_to_vector(new_wb.bis, 1)
    # Assign to each face a case number
    my_temp_list = [0]*len(my_face_list)
    # After each round, I will always sort the elements so their codimensions are small to large.
    for j in range(len(my_temp_list)):
        # If the equations defining F_j span the new equation
        face_equs = [my_wbs[ind].bis for ind in my_face_list[j].equs]
        face_vecs = [word_to_vector(equ, 1) for equ in face_equs]
        if in_span(extract_basis(face_vecs), new_vec):
            my_temp_list[j] = 1
        # If the face is a minimal face. Since excellent me always sorts the faces, I can always check j from small to large.
        elif my_face_list[j].subfaces == []:
            # if the new hyperplane intersects with the minimal face, it's type 6
            if find_positive_definite_intersection(face_equs + [new_wb.bis]).is_intersection:
                my_temp_list[j] = 6
            # if the new hyperplane does not intersect with the minimal face, it's type 2 or 4
            else:
                face_sample_point = my_face_list[j].sample_point
                if np.trace(face_sample_point @ new_wb.bis) > 0:
                    my_temp_list[j] = 2
                else:
                    my_temp_list[j] = 4
        # If the face has subfaces.
        else:
            face_subfaces = my_face_list[j].subfaces
            face_subfaces_temp = [my_temp_list[ind] for ind in face_subfaces]
            # If the type of either subface is 6.
            if 6 in face_subfaces_temp:
                my_temp_list[j] = 6
            # If the type of a subface is 2 or 3, while which of the other subface is 4 or 5.
            elif {2, 3} & set(face_subfaces_temp) and {4, 5} & set(face_subfaces_temp):
                my_temp_list[j] = 6
            # If the type of a subface is 1, 3, or 5.
            elif {1, 3, 5} & set(face_subfaces_temp):
                face_sample_point = my_face_list[j].sample_point
                if np.trace(face_sample_point @ new_wb.bis) > 0:
                    my_temp_list[j] = 3
                else:
                    my_temp_list[j] = 5
            # Types of all subfaces are 2, or are 4.
            else:
                # If the new hyperplane intersects the span of the face
                if find_positive_definite_intersection(face_equs + [new_wb.bis]).is_intersection:
                    # Sample point of this intersection
                    face_inters_sample_point = find_positive_definite_intersection(face_equs + [new_wb.bis]).sample_point
                    # Find out the equations shape the sides of the face 
                    face_subfaces_equs = []
                    for ind in face_subfaces:
                        if my_face_list[ind].codim == my_face_list[j].codim + 1:
                            face_subfaces_equs_temp = [elem for elem in my_face_list[ind].equs if elem not in my_face_list[j].equs]
                            face_subfaces_equs[:] = list(set(face_subfaces_equs) | set(face_subfaces_equs_temp))
                    # Assume the type is 6
                    my_temp_list[j] = 6
                    for ind in face_subfaces_equs:
                        # However, if any side separates the sample point from the face, the type is either 2 or 4
                        if np.trace(face_inters_sample_point @ my_wbs[ind].bis) < 0:
                            my_temp_list[j] = face_subfaces_temp[0]
                            break
                # If the new hyperplane does not intersect the span of the face, the type is either 2 or 4
                else:
                    my_temp_list[j] = face_subfaces_temp[0]
    # If the face is of type 4 or 5, it will be deleted.
    ind_remove_list = [j for j in range(len(my_temp_list)) if my_temp_list[j] in [4, 5]]
    for j in sorted(ind_remove_list, reverse=True):
        del my_face_list[j]
        del my_temp_list[j]
    # The presence of these faces in subfaces is also erased.
    for j in range(len(my_temp_list)):
        my_face_list[j].subfaces = [ind for ind in my_face_list[j].subfaces if ind not in ind_remove_list]
        List_subfaces_temp = []
        for ind in my_face_list[j].subfaces:
            decrease = sum(1 for val in ind_remove_list if val < ind)
            List_subfaces_temp.append(ind - decrease)
        my_face_list[j].subfaces = List_subfaces_temp.copy()
    # Update the remaining elements
    for j in range(len(my_temp_list)):
        # If the face is of type 1, the new equation will be added.
        if my_temp_list[j] == 1:
            my_face_list[j].equs.append(len(my_wbs))
        # If the face is of type 6:
        elif my_temp_list[j] == 6:
            # Equations for new face
            new_face_equs = my_face_list[j].equs + [len(my_wbs)]
            # Codimension of new face
            new_face_codim = my_face_list[j].codim + 1
            # Subfaces for both old and new faces
            new_face_subfaces = [ind for ind in my_face_list[j].subfaces if my_temp_list[ind] == 1]
            for ind in my_face_list[j].subfaces:
                if ind < len(my_temp_list):
                    if my_temp_list[ind] == 6:
                        my_face_list[j].subfaces.append(my_face_list[ind].subfaces[-1])
                        new_face_subfaces.append(my_face_list[ind].subfaces[-1])
            my_face_list[j].subfaces.append(len(my_face_list))
            # Sample point for the new face
            if len(new_face_subfaces) == 0:
                face_equs = [my_wbs[ind].bis for ind in my_face_list[j].equs]
                new_face_sample_point = find_positive_definite_intersection(face_equs + [new_wb.bis]).sample_point
            elif len(new_face_subfaces) >= 2:
                new_face_sample_point = sum((my_face_list[ind].sample_point for ind in new_face_subfaces), np.zeros((3, 3)))
                new_face_sample_point = new_face_sample_point/((det(new_face_sample_point)) ** (1/3))
            # If only one subface
            else:
                face_equs = [my_wbs[ind].bis for ind in my_face_list[j].equs]
                # subface_equ_ind = first_unique_element(my_face_list[new_face_subfaces[0]].equs, new_face_equs)
                subface_equ_ind = next((ind for ind in my_face_list[new_face_subfaces[0]].equs if ind not in set(new_face_equs)), None)
                subface_equ = my_wbs[subface_equ_ind].bis
                subface_sample_point = my_face_list[new_face_subfaces[0]].sample_point
                new_face_sample_point = perturb_within_plane(subface_sample_point, face_equs + [new_wb.bis], subface_equ)
            # Sample point for the old face
            if np.trace(my_face_list[j].sample_point @ new_wb.bis) < np.sqrt(a_tol):
                face_equs = [my_wbs[ind].bis for ind in my_face_list[j].equs]
                old_face_sample_point = perturb_within_plane(new_face_sample_point, face_equs, new_wb.bis)
                # Check if this point is inside the polytope. 
                # The new face will be good. Moreover, if the old sample point is on the face, it will be fine.
                # To make it safe, add that ind is not in my_face_list[j].equs
                while any(ind not in my_face_list[j].equs and np.trace(wb.bis @ my_face_list[j].sample_point) > a_tol\
                          and np.trace(wb.bis @ old_face_sample_point) < a_tol for ind, wb in enumerate(my_wbs)):
                    old_face_sample_point = 0.5*(old_face_sample_point + new_face_sample_point)
                temporary_sample_point = 0.5*(old_face_sample_point + new_face_sample_point)
                temporary_sample_point = temporary_sample_point/((det(temporary_sample_point)) ** (1/3))
                my_face_list[j].sample_point = temporary_sample_point
            # Save the new face to my_face_list
            my_face_list.append(Poly_Face(new_face_equs, new_face_codim, new_face_subfaces, np.array(new_face_sample_point)))
    # Save the new equation to bises_active
    my_wbs.append(new_wb)
    # Remove the unnecessary equations
    equ_remove_list = list(range(len(my_wbs)))
    for j in range(len(my_face_list)):
        if my_face_list[j].codim == 1:
            equ_remove_list = [ind for ind in equ_remove_list if ind != my_face_list[j].equs[0]]
    for j in sorted(equ_remove_list, reverse=True):
        del my_wbs[j]
    for j in range(len(my_face_list)):
        my_face_list[j].equs = [ind for ind in my_face_list[j].equs if ind not in equ_remove_list]
        List_equs_temp = []
        for ind in my_face_list[j].equs:
            decrease = sum(1 for val in equ_remove_list if val < ind)
            List_equs_temp.append(ind - decrease)
        my_face_list[j].equs = List_equs_temp.copy()
    # Sort the faces again, including the subfaces
    my_face_list_indexed = [(i, face) for i, face in enumerate(my_face_list)]
    my_face_list_indexed.sort(key=lambda obj: obj[1].codim, reverse=True)
    index_mapping = {old_index: new_index for new_index, (old_index, _) in enumerate(my_face_list_indexed)}
    for _, face in my_face_list_indexed:
        face.subfaces = [index_mapping[ind] for ind in face.subfaces]
    my_face_list = [face for _, face in my_face_list_indexed]
    return my_wbs, my_face_list

# Check if two given faces are paired by a given word.
# The bisectors are described by my_wbs. The polytope structure is described by my_face_list.
def face_is_paired(my_wbs, my_face_list, old_face_ind, new_face_ind, word):
    if my_face_list[old_face_ind].codim != my_face_list[new_face_ind].codim:
        return False
    if len(my_face_list[old_face_ind].subfaces) != len(my_face_list[new_face_ind].subfaces):
        return False
    old_face_equations = [my_wbs[ind].bis for ind in my_face_list[old_face_ind].equs]
    new_face_equations = [my_wbs[ind].bis for ind in my_face_list[new_face_ind].equs]
    if not equal_spaces(old_face_equations, new_face_equations, word):
        return False
    if len(my_face_list[old_face_ind].subfaces) == 0:
        return True
    cod = my_face_list[old_face_ind].codim
    old_facets = [j for j in my_face_list[old_face_ind].subfaces if my_face_list[j].codim == cod + 1]
    new_facets = [k for k in my_face_list[new_face_ind].subfaces if my_face_list[k].codim == cod + 1]
    if len(old_facets) != len(new_facets):
        return False
    for j in old_facets:
        if not any(face_is_paired(my_wbs, my_face_list, j, k, word) for k in new_facets):
            return False
    return True

# Find unpaired ridges in a non-exact (pre-)Dirichlet-Selberg domain
def unpaired_ridge(my_wbs, my_face_list):
    facet_indices = [i for i in range(len(my_face_list)) if my_face_list[i].codim == 1]
    unpaired_ridges = []
    for i in facet_indices:
        i_ridges = [j for j in my_face_list[i].subfaces if my_face_list[j].codim == 2]
        i_pair = next((i_0 for i_0 in facet_indices if np.all(np.abs(my_wbs[my_face_list[i].equs[0]].word @ my_wbs[my_face_list[i_0].equs[0]].word\
                                                                     - np.eye(3))<a_tol)), None)
        if i_pair != None:
            i_pair_ridges = [j_0 for j_0 in my_face_list[i_pair].subfaces if my_face_list[j_0].codim == 2]
            for j in i_ridges:
                j_equations = [my_wbs[ind].bis for ind in my_face_list[j].equs]
                j_pair = next((j_0 for j_0 in i_pair_ridges if equal_spaces(j_equations,\
                                        [my_wbs[ind].bis for ind in my_face_list[j_0].equs],my_wbs[my_face_list[i].equs[0]].word)), None)
                if j_pair == None:
                    mat = (my_wbs[my_face_list[i].equs[0]].word).T @ my_face_list[j].sample_point @ my_wbs[my_face_list[i].equs[0]].word
                    if all(np.trace(mat @ my_wbs[ind].bis)> a_tol for ind in range(len(my_wbs)) if ind not in my_face_list[i_pair].equs):
                        unpaired_ridges.append([i, j])
    return unpaired_ridges

# Find the word that takes the destination point into the Dirichlet-Selberg domain
def path_word(my_wbs, my_face_list, dest_point):
    if not is_positive_definite(dest_point):
        raise ValueError("path_word: destination point must be positive definite.")
    dest_point = dest_point/(det(dest_point) ** (1/3))                            # Normalize the destination point
    center = next(face.sample_point for face in my_face_list if face.codim == 0)  # The center is the sample point of the polytope itself
    if all(np.trace(dest_point @ wb.bis)>-a_tol for wb in my_wbs):                  # If the destination is already in the Dirichlet-Selberg domain
        return []
    meet_ind = max(range(len(my_wbs)), key = lambda i: -(np.trace(dest_point @ my_wbs[i].bis))/(np.trace(center @ my_wbs[i].bis)))
    meet_word = my_wbs[meet_ind].word                                             # The word corresponding to the first bisector meeting the ray from the center towards the destination
    new_dest = meet_word.T @ dest_point @ meet_word                               # Take the destination closer to the center
    new_path = path_word(my_wbs, my_face_list, new_dest)
    new_path.insert(0, meet_ind)
    return new_path


Core solver functions

In [7]:
# Compute the polytope structure of the Dirichlet-Selberg domain from generators and center.
# The algorithm will thoroughly try all words up to length_1. After this, it searches for unpaired ridges and try to add any words up to length_2 that can pair them.
# The algorithm will stop trying after a given loop times eliminating the unpaired ridges.
def compute_selberg_domain(generators, length_1, length_2, loop_times, center):
    wbs = word_bisectors(generators, length_1, center)
    more_wbs = word_bisectors(generators, length_2, center)
    my_wbs = []                                           # Initialize the word-bisectors used in the polytope
    my_face_list = [Poly_Face([], 0, [], np.array(center))]     # Initialize the polytope, which is just the entire space X_3
    for i in tqdm(range(len(wbs)), desc ="Adding words to the Dirichlet-Selberg domain"):
        my_wbs, my_face_list = selberg_domain_add_facet(my_wbs, my_face_list, wbs[i]) # Add the i-th word-bisector to the polytope
    for _ in tqdm(range(loop_times), desc ="Adding more words to eliminate unpaired ridges"):
        unpaired_ridges = unpaired_ridge(my_wbs, my_face_list)
        if not unpaired_ridges:                           # Stop searching if all ridges are paired
            break
        else:
            facet_indices = [i for i in range(len(my_face_list)) if my_face_list[i].codim == 1]
            # print("current number of facets:", len(facet_indices))
            # print("current number of unpaired ridges", len(unpaired_ridges))
            min_dist = np.inf
            new_wb = None                                     # Searching for the candidate bisector closest to the origin
            for i, j in unpaired_ridges:
                i_pair = next((i_0 for i_0 in facet_indices if np.all(np.abs(my_wbs[my_face_list[i].equs[0]].word @ my_wbs[my_face_list[i_0].equs[0]].word\
                                                                     - np.eye(3))<a_tol)), None)
                j_equations = [my_wbs[ind].bis for ind in my_face_list[j].equs]
                i_pair_equation = my_wbs[my_face_list[i_pair].equs[0]].bis
                for k in range(len(more_wbs)):
                    if np.trace(more_wbs[k].bis @ np.array(center)) < min_dist:
                        candidate_equations = [i_pair_equation, more_wbs[k].bis]
                        if equal_spaces(j_equations, candidate_equations, my_wbs[my_face_list[i].equs[0]].word):
                            min_dist = np.trace(more_wbs[k].bis @ np.array(center))
                            new_wb = more_wbs[k]              # Update the desired bisector if a smaller distance is detected
            if new_wb is not None:
                my_wbs, my_face_list = selberg_domain_add_facet(my_wbs, my_face_list, new_wb)
            # print("ridge fixed")
    return my_wbs, my_face_list

# The short version of the Dirichlet-Selberg domain computing algorithm
def compute_selberg_domain_short(generators, length, center):
    my_wbs, my_face_list = compute_selberg_domain(generators, length, length, 0, center)
    return my_wbs, my_face_list

# Check if a polytope is exact with respect to the canonical facet pairings
def polytope_is_exact(my_wbs, my_face_list):
    facet_indices = [i for i in range(len(my_face_list)) if my_face_list[i].codim == 1] # Get a list of facets
    paired_indices = []
    for i in facet_indices:
        i_pair = next((j for j in facet_indices if face_is_paired(my_wbs, my_face_list,\
                                                                  i, j, my_wbs[my_face_list[i].equs[0]].word)), None)
        if i_pair == None:
            return False, facet_indices, None                                           # False if facets are not paired
        else:
            paired_indices.append(i_pair)
    return True, facet_indices, paired_indices                                          # Corresponding facets are canonically paired

# Compute the ridge cycles for a given exact polytope in X_3. Will rewrite to utilize polytope_is_exact() more
def compute_ridge_cycle(my_wbs, my_face_list):
    is_exact, facet_indices, paired_indices = polytope_is_exact(my_wbs, my_face_list)
    if not is_exact:                  # Ridges cycles are defined only for exact polytopes 
        return None
    all_ridge_indices = [i for i in range(len(my_face_list)) if my_face_list[i].codim == 2]
    ridge_cycle_list = []                                        # Initialize the list of ridge cycles
    for i in facet_indices:
        ridge_indices = [j for j in my_face_list[i].subfaces if j in all_ridge_indices]
        for j in ridge_indices:                                  # Consider the index pair for a facet and a ridge of it
            if any(j in ridge_cycle.ridge for ridge_cycle in ridge_cycle_list):
                continue                                         # Case if it is already in a ridge cycle
            current_ridge = j
            current_facet = i                                # Chasing the ridges along the cycle
            current_pairing = my_face_list[current_facet].equs[0]
            ridge_cycle = Ridge_Cycle([current_ridge], [current_pairing])
            for _ in range(2*len(all_ridge_indices)):        # Ridge cycles will not be too long
                mapped_facet = next((mapped_i for mapped_i in facet_indices if \
                                    face_is_paired(my_wbs, my_face_list, current_facet, mapped_i, my_wbs[current_pairing].word)), None)
                new_ridge_indices = [new_j for new_j in my_face_list[mapped_facet].subfaces if new_j in all_ridge_indices]
                new_ridge = next((new_j for new_j in new_ridge_indices if \
                                    face_is_paired(my_wbs, my_face_list, current_ridge, new_j, my_wbs[current_pairing].word)), None)
                new_facet = next(new_i for new_i in facet_indices if new_ridge in my_face_list[new_i].subfaces and new_i != mapped_facet)
                new_pairing = my_face_list[new_facet].equs[0]
                if new_ridge == ridge_cycle.ridge[0] and new_pairing == ridge_cycle.pairing[0]:
                    ridge_cycle_list.append(ridge_cycle)      # Add the ridge cycle to ridge_cycle_list if a full cycle is obtained
                    break
                ridge_cycle.ridge.append(new_ridge)       # Shift to the next ridge and facet if the cycle is not completed
                ridge_cycle.pairing.append(new_pairing)
                current_ridge = new_ridge
                current_facet = new_facet
                current_pairing = new_pairing
    return ridge_cycle_list

# Compute the angle sum for a ridge cycle of a given polytope in X_3
# Specifically, the result is a natural number k if the angle sum is 2pi/k, and is None if the ridge cycle does not satisfy this angle sum condition
def angle_sum(my_wbs, my_face_list, ridge_cycle):
    angle_sum = 0                                           # Initialize the angle sum
    point = my_face_list[ridge_cycle.ridge[0]].sample_point # The base point of the first ridge is selected to be the given sample point
    for i in range(len(ridge_cycle.ridge)):
        first_bis = my_face_list[ridge_cycle.ridge[i]].equs[0]
        second_bis = my_face_list[ridge_cycle.ridge[i]].equs[1]
        angle = Riemannian_angle(my_wbs[first_bis].bis, my_wbs[second_bis].bis, point) # Compute the Riemannian angle between the bisectors
        angle_sum = angle_sum + angle                       # Add this to the angle sum
        word = my_wbs[ridge_cycle.pairing[i]].word          
        point = word.T @ point @ word                       # Shift to the paired base point of the next ridge
    quotient = 2*np.pi/angle_sum                            # Check if the quotient of the angle sum with 2pi is a natural number
    quotient_round = round(quotient)
    if abs(quotient - quotient_round)> 100*a_tol:
        return None
    else:
        return quotient_round

# Test if a given matrix in SL(3,R) is recovered by the facet pairings of the domain
def word_is_recovered(my_wbs, my_face_list, matrix):
    matrix = matrix/(det(matrix)**(1/3))
    center = next(face.sample_point for face in my_face_list if face.codim == 0)  # The center is the sample point of the polytope itself
    dest_point = matrix.T @ center @ matrix
    word = path_word(my_wbs, my_face_list, dest_point)
    for ind in word:
        matrix = matrix @ my_wbs[ind].word
    if np.all(np.abs(matrix - np.eye(3))<a_tol):
        return True
    else:
        return False

Demo Program: A congruence subgroup of level-two in SL(3,Z) (may take several minutes to finish)

In [8]:
generators = []
for i, j in itertools.permutations(range(3), 2):
    generator = np.eye(3)
    generator[i, j] = 2
    generators.append(generator)
center = np.eye(3)
my_wbs, my_face_list = compute_selberg_domain(generators, 1, 2, 20, center)
# Count the number of faces
print("Number of facets:", sum(1 for face in my_face_list if face.codim == 1))
print("Number of ridges:", sum(1 for face in my_face_list if face.codim == 2))
print("Number of peaks:", sum(1 for face in my_face_list if face.codim == 3))
print("Number of edges:", sum(1 for face in my_face_list if face.codim == 4))
print("Number of vertices:", sum(1 for face in my_face_list if face.codim == 5))

Adding words to the Dirichlet-Selberg domain:   0%|          | 0/12 [00:00<?, ?it/s]

/usr/local/python/3.12.1/lib/python3.12/site-packages/cvxpy/problems/problem.py:1510: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


Adding more words to eliminate unpaired ridges:   0%|          | 0/20 [00:00<?, ?it/s]

Number of facets: 24
Number of ridges: 84
Number of peaks: 96
Number of edges: 0
Number of vertices: 0


Demo Program: A lattice group in SL(3,Z[1/2]) with 5-simplex fundamental domain (quick)

In [12]:
generator = np.array([[1/2, 1/2, 0],
        [1/2, -1/2, 1],
        [1/2, -1/2, -1]])
generators = [generator, generator[[1, 2, 0]][:, [1, 2, 0]]]
center = np.eye(3)
my_wbs, my_face_list = compute_selberg_domain(generators, 1, 3, 20, center)
# Count the number of faces
print("Number of facets:", sum(1 for face in my_face_list if face.codim == 1))
print("Number of ridges:", sum(1 for face in my_face_list if face.codim == 2))
print("Number of peaks:", sum(1 for face in my_face_list if face.codim == 3))
print("Number of edges:", sum(1 for face in my_face_list if face.codim == 4))
print("Number of vertices:", sum(1 for face in my_face_list if face.codim == 5))

Adding words to the Dirichlet-Selberg domain:   0%|          | 0/4 [00:00<?, ?it/s]

Adding more words to eliminate unpaired ridges:   0%|          | 0/20 [00:00<?, ?it/s]

Number of facets: 6
Number of ridges: 15
Number of peaks: 16
Number of edges: 0
Number of vertices: 0


Verify that the sample points are taken correctly

In [13]:
# Make sure every sample point lies on the corresponding plane. Expected to be something close to zero.
max_trace = 0
for face in my_face_list:
    for ind in face.equs:
        my_trace = np.trace(my_wbs[ind].bis @ face.sample_point)
        max_trace = max(max_trace, abs(my_trace))
print("The largest trace for sample points multiplying with corresponding facets:", max_trace)
# Make sure every sample point is in the interior. Expected to be a positive number.
min_trace = np.inf
for face in my_face_list:
    for ind in range(len(my_wbs)):
        if ind not in face.equs:
            my_trace = np.trace(my_wbs[ind].bis @ face.sample_point)
            min_trace = min(min_trace, my_trace)
print("The smallest trace of sample points multiplying with non-corresponding facets:", min_trace)

The largest trace for sample points multiplying with corresponding facets: 2.7755575615628914e-15
The smallest trace of sample points multiplying with non-corresponding facets: 0.0875728471814019


Compute for the ridge cycles

In [14]:
is_exact, original_facets, paired_facets = polytope_is_exact(my_wbs, my_face_list)
if not is_exact:
    print("The Dirichlet-Selberg domain is not exact.")
else:
    ridge_cycles = compute_ridge_cycle(my_wbs, my_face_list)
    for i in range(len(ridge_cycles)):
        ridge_cycle = ridge_cycles[i]
        print("The indices of ridges in the", i, "th cycle:", ridge_cycle.ridge)
        pairings = [my_wbs[wb].word for wb in ridge_cycle.pairing]
        # word = np.eye(3)
        # for pairing in pairings:
        #     word = word @ pairing
        # print("The word generated from the", i, "th ridge cycle:")
        # print(word)
        my_angle_sum = angle_sum(my_wbs, my_face_list, ridge_cycle)
        if my_angle_sum == None:
            print("The", i, "th ridge cycle does not satisfy the angle sum condition.")
        else:
            print("The angle sum divisor for the", i, "th ridge cycle equals", my_angle_sum)

The indices of ridges in the 0 th cycle: [16, 17, 21]
The angle sum divisor for the 0 th ridge cycle equals 2
The indices of ridges in the 1 th cycle: [19, 24, 27]
The angle sum divisor for the 1 th ridge cycle equals 2
The indices of ridges in the 2 th cycle: [22, 28, 30]
The angle sum divisor for the 2 th ridge cycle equals 2
The indices of ridges in the 3 th cycle: [26, 18, 25]
The angle sum divisor for the 3 th ridge cycle equals 1
The indices of ridges in the 4 th cycle: [20, 29, 23]
The angle sum divisor for the 4 th ridge cycle equals 2


Verify if each generator is recovered as a product of the facet pairings

In [15]:
for ind, matrix in enumerate(generators):
    if word_is_recovered(my_wbs, my_face_list, matrix):
        print("the", ind, "th generator is recovered by the product of facet pairings with indices:", path_word(my_wbs, my_face_list, matrix.T @ center @ matrix))
    else:
        print("the", ind, "th generator is not recovered by the facet pairings." )

the 0 th generator is recovered by the product of facet pairings with indices: [2]
the 1 th generator is recovered by the product of facet pairings with indices: [3]
